<a href="https://colab.research.google.com/github/ekanshojha/Credit-Card-Fraud-Detection/blob/main/Image_Classification_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as tf
device=torch.device("cuda:0")


In [ ]:
transform=tf.Compose([tf.ToTensor(),tf.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [ ]:
traindata=torchvision.datasets.CIFAR10(root="./data",train=True,download=True,transform=transform)
testdata=torchvision.datasets.CIFAR10(root="./data",train=False,download=True,transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 43413153.58it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
trainloader=torch.utils.data.DataLoader(traindata,batch_size=40,shuffle=False)
testloader=torch.utils.data.DataLoader(testdata,batch_size=40,shuffle=False)

In [ ]:
classes=['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
import torchvision.models as models

In [ ]:
net=models.mobilenetv3.mobilenet_v3_small()

In [ ]:
net.to(device)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [ ]:
import torch.optim as optim
optimizer=optim.Adam(net.parameters(),lr=0.001)
criteria=torch.nn.CrossEntropyLoss()
numepochs=5
for epoch in range(numepochs):
  for images,labels in trainloader:
    images=images.to(device)
    labels=labels.to(device)
    optimizer.zero_grad()
    output=net(images)
    loss=criteria(output,labels)
    loss.backward()
    optimizer.step()
  print("Epoch: ",epoch,"->Loss:",loss.item()/len(images))

Epoch:  0 ->Loss: 0.038607043027877805
Epoch:  1 ->Loss: 0.03220315575599671
Epoch:  2 ->Loss: 0.030798548460006715
Epoch:  3 ->Loss: 0.028826630115509032
Epoch:  4 ->Loss: 0.030933290719985962


In [ ]:
total=0
correct=0
with torch.no_grad():
  for images,labels in testloader:
    images=images.to(device)
    labels=labels.to(device)
    output=net(images)
    _,predlabel=torch.max(output,1)
    total=total+40
    correct+=torch.sum(predlabel==labels.data)

print("Total : ",total," Correct: ", correct," accuracy: ",correct/total)

    # print("original:",labels)
    # print("Predicted:",predlabel)



Total :  10000  Correct:  tensor(5576, device='cuda:0')  accuracy:  tensor(0.5576, device='cuda:0')


In [ ]:
batch=iter(testloader)
im,lb=next(batch)
im=im.to(device)
lb=lb.to(device)
out=net(im)
_,pred=torch.max(out,1)
print("original: ", classes[lb[0]]," predicted: ",classes[pred[0]])


original:  cat  predicted:  cat
